In [187]:
include("src/dispersion.jl")
using Combinatorics
using LinearAlgebra

In [326]:
Nk = 6
D = 3
tsc = if D == 2 0.5 else 0.40824829046386301636 end
ki, kg = gen_kGrid(Nk, D; min = 0, max = 2π)
ki2, kg2 = gen_kGrid(Nk, D; min = -π, max = π)
qi, qg = gen_kGrid(Int(Nk/2)+1, D; min = 0, max = π, include_min=true);
ek = squareLattice_ekGrid(kg, tsc)
eq = squareLattice_ekGrid(qg, tsc)
ek2 = squareLattice_ekGrid(kg2, tsc)

kg_cut = cut_mirror(collect(kg))
ek_cut = cut_mirror(collect(ek))
ki_cut = cut_mirror(collect(ki))
ki_red, kg_red, ek_red = reduce_kGrid.((ki, kg, ek));
ki2_red, kg2_red, ek2_red = reduce_kGrid.((ki2, kg2, ek2));
qi_red, qg_red, eq_red = reduce_kGrid.((qi, qg, eq));
kg_rcut = reshape(collect(kg_red)[1:length(collect(qg_red))], size(collect(qg_red))...)
ki_rcut = reshape(collect(ki_red)[1:length(collect(qg_red))], size(collect(qg_red))...);
ek_rcut = reshape(collect(ek_red)[1:length(collect(qg_red))], size(collect(qg_red))...);
kg2_rcut = reshape(collect(kg2_red)[1:length(collect(qg_red))], size(collect(qg_red))...)
ki2_rcut = reshape(collect(ki2_red)[1:length(collect(qg_red))], size(collect(qg_red))...);
ek2_rcut = reshape(collect(ek2_red)[1:length(collect(qg_red))], size(collect(qg_red))...);
ki_red_exp = reshape(expand_kGrid(ki_red, collect(ek_red)[:]), size(ek)...);

In [331]:

function expand_kGrid(reducedInd, reducedArr::Array{T,1}) where T <: Number
     D = length(reducedInd[1])
     Nk = maximum(maximum.(reducedInd))
     newArr = Array{eltype(reducedArr)}(undef, (Nk*ones(Int64, D))...)
     for (ri,redInd) in enumerate(reducedInd)
         perms = unique(collect(permutations(redInd)))
         for p in perms
             newArr[p...] = reducedArr[ri]
          end
     end
     return newArr
end

function cut_mirror(arr)
    res = nothing
    Nk_cut = Int(size(arr,1)/2) + 1
    if ndims(arr) == 2
        res = arr[1:Nk_cut, 1:Nk_cut]
    elseif ndims(arr) == 3
        res = arr[1:Nk_cut, 1:Nk_cut, 1:Nk_cut]
    else
        println(stderr, "Error trying to reduce grid! Number of dimensions not recognized")
    end
    return res
end

function expand_mirror(arr)
    res = nothing
    Nk_old = size(arr,1)
    Nk_expand = Nk_old - 2
    size_new = size(arr) .+ Nk_expand
    
    res = Array{eltype(arr)}(undef, size_new...)
    if ndims(arr) == 2
        res[1:Nk_old,1:Nk_old] = arr
        res[Nk_old+1:end,1:Nk_old] = res[Nk_old-1:-1:2,1:Nk_old]
        res[1:end,Nk_old+1:end] = res[1:end,Nk_old-1:-1:2]
        for (i,ai) in enumerate((Nk_old-1):-1:2)
            res[Nk_old+i,Nk_old+i] = arr[ai,ai]
        end
    elseif ndims(arr) == 3
        res[1:Nk_old,1:Nk_old,1:Nk_old] = arr
        res[Nk_old+1:end,1:Nk_old,1:Nk_old] = res[Nk_old-1:-1:2,1:Nk_old,1:Nk_old]
        res[1:end,Nk_old+1:end,1:Nk_old] = res[1:end,Nk_old-1:-1:2,1:Nk_old]
        for (i,ai) in enumerate((Nk_old-1):-1:2)
            res[Nk_old+i,Nk_old+i,1:Nk_old] = arr[ai,ai,1:Nk_old]
        end
        res[1:end,1:end,Nk_old+1:end] .= res[1:end,1:end,Nk_old-1:-1:2]
        for (i,ai) in enumerate((Nk_old-1):-1:2)
            res[Nk_old+i,Nk_old+i,Nk_old+i] = arr[ai,ai,ai]
        end
    else
        println(stderr, "Error trying to reduce grid! Number of dimensions not recognized")
    end
    return res
end

expand_mirror (generic function with 2 methods)

In [332]:
collect(ki)

6×6×6 Array{Tuple{Int64,Int64,Int64},3}:
[:, :, 1] =
 (1, 1, 1)  (1, 2, 1)  (1, 3, 1)  (1, 4, 1)  (1, 5, 1)  (1, 6, 1)
 (2, 1, 1)  (2, 2, 1)  (2, 3, 1)  (2, 4, 1)  (2, 5, 1)  (2, 6, 1)
 (3, 1, 1)  (3, 2, 1)  (3, 3, 1)  (3, 4, 1)  (3, 5, 1)  (3, 6, 1)
 (4, 1, 1)  (4, 2, 1)  (4, 3, 1)  (4, 4, 1)  (4, 5, 1)  (4, 6, 1)
 (5, 1, 1)  (5, 2, 1)  (5, 3, 1)  (5, 4, 1)  (5, 5, 1)  (5, 6, 1)
 (6, 1, 1)  (6, 2, 1)  (6, 3, 1)  (6, 4, 1)  (6, 5, 1)  (6, 6, 1)

[:, :, 2] =
 (1, 1, 2)  (1, 2, 2)  (1, 3, 2)  (1, 4, 2)  (1, 5, 2)  (1, 6, 2)
 (2, 1, 2)  (2, 2, 2)  (2, 3, 2)  (2, 4, 2)  (2, 5, 2)  (2, 6, 2)
 (3, 1, 2)  (3, 2, 2)  (3, 3, 2)  (3, 4, 2)  (3, 5, 2)  (3, 6, 2)
 (4, 1, 2)  (4, 2, 2)  (4, 3, 2)  (4, 4, 2)  (4, 5, 2)  (4, 6, 2)
 (5, 1, 2)  (5, 2, 2)  (5, 3, 2)  (5, 4, 2)  (5, 5, 2)  (5, 6, 2)
 (6, 1, 2)  (6, 2, 2)  (6, 3, 2)  (6, 4, 2)  (6, 5, 2)  (6, 6, 2)

[:, :, 3] =
 (1, 1, 3)  (1, 2, 3)  (1, 3, 3)  (1, 4, 3)  (1, 5, 3)  (1, 6, 3)
 (2, 1, 3)  (2, 2, 3)  (2, 3, 3)  (2, 4, 3)  (2, 5, 3)  (2, 6, 

In [333]:
cut_mirror(collect(ki))

4×4×4 Array{Tuple{Int64,Int64,Int64},3}:
[:, :, 1] =
 (1, 1, 1)  (1, 2, 1)  (1, 3, 1)  (1, 4, 1)
 (2, 1, 1)  (2, 2, 1)  (2, 3, 1)  (2, 4, 1)
 (3, 1, 1)  (3, 2, 1)  (3, 3, 1)  (3, 4, 1)
 (4, 1, 1)  (4, 2, 1)  (4, 3, 1)  (4, 4, 1)

[:, :, 2] =
 (1, 1, 2)  (1, 2, 2)  (1, 3, 2)  (1, 4, 2)
 (2, 1, 2)  (2, 2, 2)  (2, 3, 2)  (2, 4, 2)
 (3, 1, 2)  (3, 2, 2)  (3, 3, 2)  (3, 4, 2)
 (4, 1, 2)  (4, 2, 2)  (4, 3, 2)  (4, 4, 2)

[:, :, 3] =
 (1, 1, 3)  (1, 2, 3)  (1, 3, 3)  (1, 4, 3)
 (2, 1, 3)  (2, 2, 3)  (2, 3, 3)  (2, 4, 3)
 (3, 1, 3)  (3, 2, 3)  (3, 3, 3)  (3, 4, 3)
 (4, 1, 3)  (4, 2, 3)  (4, 3, 3)  (4, 4, 3)

[:, :, 4] =
 (1, 1, 4)  (1, 2, 4)  (1, 3, 4)  (1, 4, 4)
 (2, 1, 4)  (2, 2, 4)  (2, 3, 4)  (2, 4, 4)
 (3, 1, 4)  (3, 2, 4)  (3, 3, 4)  (3, 4, 4)
 (4, 1, 4)  (4, 2, 4)  (4, 3, 4)  (4, 4, 4)

In [334]:
expand_mirror(ki_cut)

6×6×6 Array{Tuple{Int64,Int64,Int64},3}:
[:, :, 1] =
 (1, 1, 1)  (1, 2, 1)  (1, 3, 1)  (1, 4, 1)  (1, 3, 1)  (1, 2, 1)
 (2, 1, 1)  (2, 2, 1)  (2, 3, 1)  (2, 4, 1)  (2, 3, 1)  (2, 2, 1)
 (3, 1, 1)  (3, 2, 1)  (3, 3, 1)  (3, 4, 1)  (3, 3, 1)  (3, 2, 1)
 (4, 1, 1)  (4, 2, 1)  (4, 3, 1)  (4, 4, 1)  (4, 3, 1)  (4, 2, 1)
 (3, 1, 1)  (3, 2, 1)  (3, 3, 1)  (3, 4, 1)  (3, 3, 1)  (3, 2, 1)
 (2, 1, 1)  (2, 2, 1)  (2, 3, 1)  (2, 4, 1)  (2, 3, 1)  (2, 2, 1)

[:, :, 2] =
 (1, 1, 2)  (1, 2, 2)  (1, 3, 2)  (1, 4, 2)  (1, 3, 2)  (1, 2, 2)
 (2, 1, 2)  (2, 2, 2)  (2, 3, 2)  (2, 4, 2)  (2, 3, 2)  (2, 2, 2)
 (3, 1, 2)  (3, 2, 2)  (3, 3, 2)  (3, 4, 2)  (3, 3, 2)  (3, 2, 2)
 (4, 1, 2)  (4, 2, 2)  (4, 3, 2)  (4, 4, 2)  (4, 3, 2)  (4, 2, 2)
 (3, 1, 2)  (3, 2, 2)  (3, 3, 2)  (3, 4, 2)  (3, 3, 2)  (3, 2, 2)
 (2, 1, 2)  (2, 2, 2)  (2, 3, 2)  (2, 4, 2)  (2, 3, 2)  (2, 2, 2)

[:, :, 3] =
 (1, 1, 3)  (1, 2, 3)  (1, 3, 3)  (1, 4, 3)  (1, 3, 3)  (1, 2, 3)
 (2, 1, 3)  (2, 2, 3)  (2, 3, 3)  (2, 4, 3)  (2, 3, 3)  (2, 2, 

In [335]:
td = [t[i,i] for i in (size(t,1)-1):-1:2]

4-element Array{Float64,1}:
  0.23083862038415823
 -0.5082200119825282
  0.25143998795960965
 -0.6069788036368076

In [336]:
t[ti...]

ArgumentError: ArgumentError: invalid index: (5, 5) of type Tuple{Int64,Int64}

In [185]:
t[1:end,4-1:-1:2]

6×2 Array{Float64,2}:
  1.4093   -0.16782
 -0.69623  -0.606979
  0.25144   0.148394
  0.62563   2.20671
 -0.1689    1.30387
 -0.434    -1.23324

In [149]:
ki_rcut

6-element Array{Tuple{Int64,Int64},1}:
 (1, 1)
 (2, 1)
 (2, 2)
 (3, 1)
 (3, 2)
 (3, 3)

In [150]:
ki_red

10-element Array{Tuple{Int64,Int64},1}:
 (1, 1)
 (2, 1)
 (2, 2)
 (3, 1)
 (3, 2)
 (3, 3)
 (4, 1)
 (4, 2)
 (4, 3)
 (4, 4)

In [151]:
collect(ek)

4×4 Array{Float64,2}:
  6.12323e-17  -0.5  -6.12323e-17  0.5
 -0.5          -1.0  -0.5          0.0
 -6.12323e-17  -0.5  -1.83697e-16  0.5
  0.5           0.0   0.5          1.0

In [158]:
reshape(expand_kGrid(collect(ki_red)[:], collect(ek_red)[:]), size(kg)...)

4×4 Array{Float64,2}:
  6.12323e-17  -0.5  -6.12323e-17  0.5
 -0.5          -1.0  -0.5          0.0
 -6.12323e-17  -0.5  -1.83697e-16  0.5
  0.5           0.0   0.5          1.0

In [160]:
ki_red_exp

4×4 Array{Float64,2}:
  6.12323e-17  -0.5  -6.12323e-17  0.5
 -0.5          -1.0  -0.5          0.0
 -6.12323e-17  -0.5  -1.83697e-16  0.5
  0.5           0.0   0.5          1.0